### 数据处理与特征提取

在 `kick_samples` 文件夹中有 8 个子目录，分别对于不同 kick 样本的类别。采用独热编码，编码为长度为 8 的向量。

- 首先通过 `librosa` 加载音频文件

In [4]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split

def load_audio(file_path):
    # 加载音频文件
    y, sr = librosa.load(file_path)
    return y, sr

def extract_features(y, sr):
    # 提取特征
    n_fft = min(2048, len(y))
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=n_fft).T
    return mfccs

def preprocess_data(label_dir):
    X, y = [], []
    for label in os.listdir(label_dir):
        for file_name in os.listdir(os.path.join(label_dir, label)):
            audio, sr = load_audio(os.path.join(label_dir, label, file_name))
            features = extract_features(audio, sr)
            X.append(features)
            y.append(label)
    return X, y

X, y = preprocess_data('kick_samples')

### 构建模型

In [5]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, Flatten

def build_multi_task_model(input_shape, num_classes):
    # 输入层
    inputs = Input(shape=input_shape)
    
    # 共享特征提取层
    lstm1 = LSTM(128, return_sequences=True)(inputs)
    dropout1 = Dropout(0.5)(lstm1)
    lstm2 = LSTM(64)(dropout1)
    dropout2 = Dropout(0.5)(lstm2)

    # 将 LSTM 输出展平，以便与 Dense 层连接
    flattened = Flatten()(dropout2)
    
    # 时间预测分支
    cat_output = Dense(num_classes, activation='softmax', name='cat')(flattened)
    
    # 构建模型
    model = Model(inputs=inputs, outputs=cat_output)
    
    # 编译模型
    model.compile(
        loss={'cat': 'categorical_crossentropy'},
        optimizer='adam',
        metrics={'cat': 'accuracy'}
    )
    
    return model

# 假设每个任务的类别数
num_classes = 11

# 假设输入形状
input_shape = (None, 13)  # 假设我们使用13维的MFCC特征

# 构建模型
model = build_multi_task_model(input_shape, num_classes)

### 训练模型

In [6]:
from keras.utils import to_categorical

cat_dict = {
    "Top":          0,
    "Chest":        1,
    "Signature":    2,
    "Stomp":        3,
    "Punchy":       4,
    "808s":         5,
    "Distorted":    6,
    "Psy":          7,
    "Big":          8,
    "Hardstyle":    9,
    "Stadium":      10,
}

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=42)

y_train = list(map(cat_dict.get, y_train))
y_val = list(map(cat_dict.get, y_val))

y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)

In [7]:
import tensorflow as tf

# 将X_train和X_val转换为RaggedTensor
X_train_ragged = tf.ragged.constant(X_train)
X_val_ragged = tf.ragged.constant(X_val)
y_train_cat = tf.convert_to_tensor(y_train_cat)
y_val_cat = tf.convert_to_tensor(y_val_cat)

# 创建tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_ragged, y_train_cat))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_ragged, y_val_cat))

# 批量化数据
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [12]:
# 训练模型
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset
)

Epoch 1/10
10/10 [==============================] - 2s 201ms/step - loss: 0.1743 - accuracy: 0.9528 - val_loss: 0.6595 - val_accuracy: 0.8235
Epoch 2/10
10/10 [==============================] - 2s 200ms/step - loss: 0.1496 - accuracy: 0.9528 - val_loss: 0.6396 - val_accuracy: 0.8235
Epoch 3/10
10/10 [==============================] - 2s 197ms/step - loss: 0.1617 - accuracy: 0.9528 - val_loss: 0.8708 - val_accuracy: 0.7647
Epoch 4/10
10/10 [==============================] - 2s 219ms/step - loss: 0.1452 - accuracy: 0.9560 - val_loss: 0.7900 - val_accuracy: 0.7647
Epoch 5/10
10/10 [==============================] - 2s 204ms/step - loss: 0.1016 - accuracy: 0.9780 - val_loss: 1.1281 - val_accuracy: 0.6471
Epoch 6/10
10/10 [==============================] - 2s 196ms/step - loss: 0.1327 - accuracy: 0.9623 - val_loss: 0.9934 - val_accuracy: 0.7647
Epoch 7/10
10/10 [==============================] - 2s 205ms/step - loss: 0.1462 - accuracy: 0.9560 - val_loss: 0.8883 - val_accuracy: 0.7647
Epoch 

### 通过新的数据集验证

In [13]:
# 使用模型进行预测
predictions = model.predict(val_dataset)

# 转换预测结果为类别索引
predicted_classes = np.argmax(predictions, axis=1)

# 由于验证标签已经是类别索引，直接使用
true_classes = y_val

# 计算准确率
accuracy = np.mean(predicted_classes == true_classes)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

1/1 [==============================] - 1s 1s/step
Validation Accuracy: 82.35%


### 保存训练好的模型

In [14]:
# 假设 filepath 是你想要保存模型的路径
filepath = 'test_kick_1.h5'
# 保存模型
model.save(filepath)

# 检查模型文件是否存在
if os.path.exists(filepath):
    print(f"Model saved to {filepath}")
else:
    print(f"Model save failed. File not found at {filepath}")

Model saved to test_kick_1.h5
